# Queueing System

## Basic Idea

### No Randomness

In [ ]:
# No Randomness

# Define the arrival rate and service rate
arrival_rate = 6  # customers per hour
service_rate = 8  # customers per hour

# Calculate the utilization factor
utilization_factor = arrival_rate / service_rate

# Calculate the average number of customers in the system
average_customers_in_system = utilization_factor / (1 - utilization_factor)

# Calculate the average number of customers in the queue
average_customers_in_queue = (utilization_factor**2) / (1 - utilization_factor)

# Calculate the average time a customer spends in the system
average_time_in_system = 1 / (service_rate - arrival_rate)

# Calculate the average time a customer spends waiting in the queue
average_time_in_queue = utilization_factor * average_time_in_system

# Print the results
print(f"Utilization Factor: {utilization_factor:.2f}")
print(f"Average Number of Customers in the System: {average_customers_in_system:.2f}")
print(f"Average Number of Customers in the Queue: {average_customers_in_queue:.2f}")
print(
    f"Average Time a Customer Spends in the System: {average_time_in_system:.2f} hours"
)
print(
    f"Average Time a Customer Spends Waiting in the Queue: {average_time_in_queue:.2f} hours"
)

### Randomness

In [ ]:
# With Poisson Randomness on Arrival and Service Rates

import pandas as pd
import numpy as np

# Simulation parameters
arrival_rate = 6  # customers per hour
service_rate = 8  # customers per hour
dropout_threshold = 0.8  # utilization factor threshold for dropout
dropout_probability = 0.2  # probability that a customer will drop out
simulation_hours = 10  # total hours for the simulation
time_step = 1 / 60  # time step in hours (1 minute)

# Generate time series
time_series = np.arange(0, simulation_hours, time_step)

# Initialize DataFrame
df = pd.DataFrame(
    index=time_series, columns=["new_customer", "service", "in_queue", "dropped_out"]
)
df["new_customer"] = np.random.poisson(arrival_rate * time_step, len(time_series))
df["service"] = np.random.poisson(service_rate * time_step, len(time_series))
df["in_queue"] = 0
df["dropped_out"] = 0

# Initialize variables
queue = 0
served = 0

# Run the simulation
for i in range(1, len(df)):
    # Calculate the current utilization factor
    utilization_factor = queue / (service_rate * time_step)

    # Check for dropout
    if utilization_factor > dropout_threshold:
        dropouts = np.random.binomial(df["new_customer"].iloc[i], dropout_probability)
    else:
        dropouts = 0

    # Update queue and served customers
    arrivals = df["new_customer"].iloc[i] - dropouts
    queue += arrivals
    served_customers = min(queue, df["service"].iloc[i])
    queue -= served_customers

    # Record data
    df.at[time_series[i], "in_queue"] = queue
    df.at[time_series[i], "dropped_out"] = dropouts
    served += served_customers

# Calculate average metrics
average_customers_in_queue = df["in_queue"].mean()
average_dropped_out = df["dropped_out"].mean()
average_time_in_queue = average_customers_in_queue / arrival_rate
average_time_in_system = (
    average_customers_in_queue + served / simulation_hours
) / arrival_rate

# Print the results
print(f"Average Number of Customers in the Queue: {average_customers_in_queue:.2f}")
print(f"Average Number of Dropouts per Time Step: {average_dropped_out:.2f}")
print(f"Average Time a Customer Spends in the Queue: {average_time_in_queue:.2f} hours")
print(
    f"Average Time a Customer Spends in the System: {average_time_in_system:.2f} hours"
)
print(f"Total Customers Served: {served}")

### Simulation

#### Key Components of the System

1. **Producers**: These are entities that generate items at random intervals.
2. **Consumers**: These entities process the items. Each consumer tracks the number of items processed and the total processing time.
3. **Items**: These represent the entities being produced and processed. Each item has timestamps for its creation, processing start, and processing end.
4. **Events**: These manage the timing and type of activities within the system, such as item production, processing start/end, and system shutdown.
5. **Simulation Engine**: This manages the event queue and orchestrates the simulation logic.
6. **Statistics and Visualization**: This component collects and displays key performance metrics using visualizations.


In [ ]:
import heapq
import numpy as np
import matplotlib.pyplot as plt
import string
from datetime import datetime, timedelta
from typing import List, Tuple
from enum import Enum
from datetime import datetime


class Producer:
    def __init__(self, id: int):
        self.id = id
        self.next_production_time: datetime = None


class Consumer:
    def __init__(self, id: str):
        self.id = id
        self.available_time: datetime = None
        self.current_item: Item = None
        self.items_processed: int = 0
        self.total_processing_time: timedelta = timedelta()


class Item:
    def __init__(self, id: int):
        self.id = id
        self.creation_time: datetime = None
        self.processing_start_time: datetime = None
        self.processing_end_time: datetime = None


class EventType(Enum):
    ITEM_PRODUCTION = "item_production"
    PROCESSING_START = "processing_start"
    PROCESSING_END = "processing_end"
    SYSTEM_SHUTDOWN = "system_shutdown"


class Event:
    def __init__(
        self,
        time: datetime,
        event_type: str,
        producer: Producer = None,
        consumer: Consumer = None,
        item: Item = None,
    ):
        self.time: datetime = time
        self.event_type: EventType = event_type
        self.producer: Producer = producer
        self.consumer: Consumer = consumer
        self.item: Item = item

    def __lt__(self, other: "Event") -> bool:
        return self.time < other.time


def get_random_interval(distribution: str, mean: float) -> float:
    if distribution == "exponential":
        return np.random.exponential(mean)
    elif distribution == "poisson":
        return np.random.poisson(mean)
    elif distribution == "uniform":
        return np.random.uniform(0, 2 * mean)
    elif distribution == "fixed":
        return mean
    else:  # default to normal distribution
        return max(0, np.random.normal(mean, mean / 3))


def simulate_system(
    start_time: datetime,
    end_time: datetime,
    producer_count: int,
    consumer_names: List[str],
    avg_production_interval: float,
    processing_duration: float,
    production_distribution: str = "normal",
    processing_distribution: str = "normal",
) -> None:
    current_time = start_time
    producers = [Producer(i) for i in range(producer_count)]
    consumers = [Consumer(name) for name in consumer_names]
    waiting_items: List[Item] = []
    item_id = 1
    events: List[Event] = []

    # Statistics
    queue_length_over_time: List[Tuple[datetime, int]] = []
    items_produced = 0
    items_processed = 0
    items_discarded = 0
    total_wait_time: timedelta = timedelta()

    print(f"{current_time.strftime('%H:%M')} System started")

    # Schedule first item production
    for producer in producers:
        production_time = current_time + timedelta(
            minutes=get_random_interval(
                production_distribution, avg_production_interval
            )
        )
        if production_time < end_time:
            heapq.heappush(
                events,
                Event(production_time, EventType.ITEM_PRODUCTION, producer=producer),
            )

    # Schedule system shutdown
    heapq.heappush(events, Event(end_time, EventType.SYSTEM_SHUTDOWN))

    while events:
        event: Event = heapq.heappop(events)
        current_time = event.time

        queue_length_over_time.append((current_time, len(waiting_items)))

        match event.event_type:
            case EventType.ITEM_PRODUCTION:
                if current_time < end_time:
                    item = Item(item_id)
                    item.creation_time = current_time
                    print(f"{current_time.strftime('%H:%M')} Item-{item.id} produced")
                    waiting_items.append(item)
                    item_id += 1
                    items_produced += 1

                    # Schedule next item production for this producer
                    next_production = current_time + timedelta(
                        minutes=get_random_interval(
                            production_distribution, avg_production_interval
                        )
                    )
                    if next_production < end_time:
                        heapq.heappush(
                            events,
                            Event(
                                next_production,
                                EventType.ITEM_PRODUCTION,
                                producer=event.producer,
                            ),
                        )

            case EventType.PROCESSING_END:
                consumer = event.consumer
                item = consumer.current_item
                item.processing_end_time = current_time
                print(
                    f"{current_time.strftime('%H:%M')} {consumer.id} finished processing Item-{item.id}"
                )
                consumer.items_processed += 1
                consumer.total_processing_time += (
                    item.processing_end_time - item.processing_start_time
                )
                consumer.current_item = None
                consumer.available_time = current_time
                items_processed += 1

            case EventType.SYSTEM_SHUTDOWN:
                # Handle waiting items at shutdown time
                for item in waiting_items:
                    print(f"{current_time.strftime('%H:%M')} Item-{item.id} discarded")
                    items_discarded += 1
                waiting_items.clear()

        # Assign waiting items to available consumers with earliest available time
        for consumer in consumers:
            if not consumer.current_item and waiting_items and current_time < end_time:
                item = waiting_items.pop(0)
                consumer.current_item = item
                item.processing_start_time = current_time
                total_wait_time += item.processing_start_time - item.creation_time
                process_time = get_random_interval(
                    processing_distribution, processing_duration
                )
                consumer.available_time = current_time + timedelta(minutes=process_time)
                print(
                    f"{current_time.strftime('%H:%M')} {consumer.id} started processing Item-{item.id}"
                )
                heapq.heappush(
                    events,
                    Event(
                        consumer.available_time,
                        EventType.PROCESSING_END,
                        consumer=consumer,
                        item=item,
                    ),
                )

    # Handle consumers finishing up after shutdown time
    while any(consumer.current_item for consumer in consumers):
        consumer = min(
            (c for c in consumers if c.current_item), key=lambda c: c.available_time
        )
        current_time = consumer.available_time
        item = consumer.current_item
        item.processing_end_time = current_time
        print(
            f"{current_time.strftime('%H:%M')} {consumer.id} finished processing Item-{item.id}"
        )
        consumer.items_processed += 1
        consumer.total_processing_time += (
            item.processing_end_time - item.processing_start_time
        )
        consumer.current_item = None
        items_processed += 1

    print(f"{current_time.strftime('%H:%M')} System shut down")

    # Calculate and print statistics
    total_time = (end_time - start_time).total_seconds() / 3600  # in hours
    avg_wait_time = (
        total_wait_time / items_processed if items_processed > 0 else timedelta()
    )

    print(f"\nSystem Statistics:")
    print(f"Total items produced: {items_produced}")
    print(f"Total items processed: {items_processed}")
    print(f"Total items discarded: {items_discarded}")
    print(f"Average wait time: {avg_wait_time}")
    print(
        f"System utilization: {sum(c.total_processing_time.total_seconds() for c in consumers) / (total_time * len(consumers) * 3600):.2%}"
    )

    for consumer in consumers:
        print(f"\n{consumer.id} Statistics:")
        print(f"Items processed: {consumer.items_processed}")
        print(
            f"Utilization: {consumer.total_processing_time.total_seconds() / (total_time * 3600):.2%}"
        )

    # Plot queue length over time
    times, queue_lengths = zip(*queue_length_over_time)
    plt.figure(figsize=(12, 6))
    plt.plot(times, queue_lengths)
    plt.title("Queue Length Over Time")
    plt.xlabel("Time")
    plt.ylabel("Queue Length")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Plot consumer utilization
    consumer_names = [c.id for c in consumers]
    utilizations = [
        c.total_processing_time.total_seconds() / (total_time * 3600) for c in consumers
    ]
    plt.figure(figsize=(10, 6))
    plt.bar(consumer_names, utilizations)
    plt.title("Consumer Utilization")
    plt.xlabel("Consumer")
    plt.ylabel("Utilization")
    plt.ylim(0, 1)
    for i, v in enumerate(utilizations):
        plt.text(i, v, f"{v:.2%}", ha="center", va="bottom")
    plt.tight_layout()
    plt.show()


# Example usage
start_time = datetime(2024, 1, 1, 9, 0)
end_time = datetime(2024, 1, 6, 21, 0)
producer_count = 2
consumer_names = [f"Consumer-{letter}" for letter in string.ascii_uppercase[:7]]
avg_production_interval = 10  # minutes
processing_duration = 30  # minutes

simulate_system(
    start_time,
    end_time,
    producer_count,
    consumer_names,
    avg_production_interval,
    processing_duration,
    production_distribution="poisson",
    processing_distribution="fixed",
)